In [1]:
from pathlib import Path
import pandas as pd

import os
from pathlib import Path
import pandas as pd
import datetime
from datetime import datetime


def import_data_from_csv() -> dict:
    """Read data from csv files (prepared by transfermrkt dataset)

    Returns:
        list[pd.DataFrame] : list of dataframes (single csv to single dataframe)
    """

    # NOTE: This wont work for jupyter so we are using sth else for now
    # # Get the absolute path of the current script (jupyter dir)
    # BASE_DIR = Path(__file__).resolve().parent
        
    # NOTE: Later when we are converting this into an actual python file, comment this line  
    # Define the base and data directories
    BASE_DIR = Path.cwd()
    DATA_DIR = BASE_DIR.parents[0] / "data" / "transfer_data"
    # CLUBS_FILE = "clubs.csv"
    # ELOS_FILE = "club_elo_2024-09-21.csv"
    # # Load CSV data
    # # fbref_df = pd.read_csv(os.path.join(DATA_DIR, FBREF_FILE), sep=',', encoding='UTF-8')
    # clubs_df = pd.read_csv(os.path.join(DATA_DIR, CLUBS_FILE), sep=",", encoding="UTF-8")
    # elos_df = pd.read_csv(os.path.join(DATA_DIR, ELOS_FILE), sep=",", encoding="UTF-8")
    # elos_df = elos_df.head(632)  # Restrict the ELO data to the first 632 rows

    # # Build the path to the data directory
    # DATA_DIR = BASE_DIR / "transfer_data"

    # BASE_DIR = Path.cwd()

    # Build the path to the data directory

    # # Example of using the path to the data directory
    # csv_file = DATA_DIR / 'your_file.csv'
    # print(csv_file)

    # import all files in Data folder and read into dataframes
    dataframes = {}

    

    # Actual reading csv flies
    for dirpath, dirname, filenames in os.walk(DATA_DIR):
        for filename in filenames:
            file = filename.split(".")
            file = file[0] + "_df"
            if file != "_df":
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath, sep=",", encoding="UTF-8")
                exec(f"{file} = df.copy()")
                print(file, df.shape)
                dataframes[file] = df.copy()
    print("Data imported")

    return dataframes

def add_season_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean up date to datetime datatype
    Add month_year, season column based on date
    @param df: 
    @return: 
    """
    # Clean up player valuations
    df_copy = df.copy()
    df_copy['date'] = pd.to_datetime(df_copy['date'])
    if 'season' not in df_copy.columns:
        # df_copy['month_year'] = df_copy['date'].apply(lambda x: f"08 {x.year}" if x >= pd.Timestamp(x.year, 7, 1)\
        #     else f"08 {x.year-1}")
        df_copy['season'] = df_copy['date'].apply(lambda x: f"{x.year}" if x >= pd.Timestamp(x.year, 7, 1) \
            else f"{x.year-1}")
    # df_copy = df_copy.loc[df_copy.groupby(['season'])['date'].idxmin()] #Get value only at start of season
    return df_copy


In [2]:
# Below is just to suppress warnings...
competitions_df = pd.DataFrame()
appearances_df = pd.DataFrame()
player_valuations_df = pd.DataFrame()
game_events_df = pd.DataFrame()
players_df = pd.DataFrame()
games_df = pd.DataFrame()
club_games_df = pd.DataFrame()
clubs_df = pd.DataFrame()
players_elo_df = pd.DataFrame()
dataframes = import_data_from_csv()

for file_name, dataframe in dataframes.items():
    exec(f"{file_name} = dataframe.copy()")
    # print(file_name)
    # print(dataframe)
    
    
    
# Clean up player valuations
# player_valuations_df['date'] = pd.to_datetime(player_valuations_df['date'])
# player_valuations_df['month_year'] = player_valuations_df['date'].apply(lambda x: f"08 {x.year}" \
#     if x >= pd.Timestamp(x.year, 7, 1) else f"08 {x.year-1}")
# player_valuations_df['season'] = player_valuations_df['date'].apply(lambda x: f"{x.year}-{x.year+1}" \
#     if x >= pd.Timestamp(x.year, 7, 1) else f"{x.year-1}-{x.year}")
# player_valuations_df = player_valuations_df.loc[player_valuations_df.groupby(['player_id','season'])['date'].idxmin()] #Get value only at start of season
# df_sorted = add_season_column(player_valuations_df)
# df_sorted = df_sorted.loc[df_sorted.groupby(['player_id','season'])['date'].idxmin()] #Get value only at start of season
# df_sorted.head()
# # From the oldest to most recent, init. player's elo based on market value.
# df_sorted = player_valuations_df.sort_values(by=['player_id', 'date'])
# oldest_player_valuations_df = df_sorted.drop_duplicates(subset='player_id', keep='first')
# df = oldest_player_valuations_df.copy()
# seasons = sorted(oldest_player_valuations_df['season'].unique())
# seasons


game_lineups_df (2191911, 10)
competitions_df (44, 11)
appearances_df (1610495, 13)
player_valuations_df (486009, 5)
game_events_df (761513, 10)
transfers_df (77585, 10)
players_df (32392, 23)
games_df (69635, 23)
club_games_df (139270, 11)
players_elo_df (237578, 26)
clubs_df (439, 17)
Data imported


In [4]:
games_df.dtypes

game_id                     int64
competition_id             object
season                      int64
round                      object
date                       object
home_club_id                int64
away_club_id                int64
home_club_goals             int64
away_club_goals             int64
home_club_position        float64
away_club_position        float64
home_club_manager_name     object
away_club_manager_name     object
stadium                    object
attendance                float64
referee                    object
url                        object
home_club_formation        object
away_club_formation        object
home_club_name             object
away_club_name             object
aggregate                  object
competition_type           object
dtype: object

In [3]:
def sort_df_by_date(df: pd.DataFrame) -> pd.DataFrame:
    """Sort dataframe by date
    It handles changing the date to datetime format as well

    Args:
        df (pd.DataFrame): Assume club_games_df have "date"

    Returns:
        pd.DataFrame: _description_
    """

    # NOTE: This is why it might fail? Chagne to index or columns if this fails
    if "date" not in df.columns:
        raise ValueError("The 'date' column does not exist in the DataFrame.")

    df["date"] = df["date"].str.strip()
    df["date"] = pd.to_datetime(df["date"], errors="coerce", format="%Y-%m-%d")
    df.dropna(subset=["date"], inplace=True)
    # Sort by date
    df = df.sort_values(by=["date"])

    return df


# games_df = sort_df_by_date(games_df)
games_df = add_season_column(games_df)
player_valuations_df = add_season_column(player_valuations_df)
# Create new df

# Create season column for players df...for each season he ran?
def init_players_elo_df(players_df: pd.DataFrame) -> pd.DataFrame:
    """
    Init players elo df based on a players_df
    @param players_df: 
    @return: 
    """
    players_elo_df = players_df.copy()
    players_elo_df['elo'] = None
    # players_elo_df['season'] = None
    # Find seasons player played for each season
    df_sorted = add_season_column(player_valuations_df)
    df_sorted = df_sorted.loc[df_sorted.groupby(['player_id','season'])['date'].idxmin()] 
    # seasons = sorted(oldest_player_valuations_df['season'].unique())
    players_elo_df = players_elo_df.merge(
    df_sorted[['player_id', 'season']], 
    on='player_id', 
    how='left'
    )

    # Get value only at start of season
    # df_sorted = player_valuations_df.sort_values(by=['player_id', 'date'])
    # oldest_player_valuations_df = df_sorted.drop_duplicates(subset='player_id', keep='first')
    # df = oldest_player_valuations_df.copy()
    
    return players_elo_df

players_elo_df = init_players_elo_df(players_df) 


In [111]:
player_valuations_df

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur,elo,season
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0,None,2004
1,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0,None,2005
2,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0,None,2006
3,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0,None,2007
4,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0,None,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237573,1306131,Fábio,Sambú,Fábio Sambú,2024,2503,fabio-sambu,Portugal,Barreiro,Portugal,...,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/fabio-sambu/pr...,PO1,Boavista Futebol Clube,NaN,NaN,None,NaN
237574,1306851,NaN,Hakim,Hakim,2024,5724,hakim,Ghana,Techiman,Ghana,...,2026-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/hakim/profil/s...,DK1,Randers Fodbold Club,NaN,NaN,None,NaN
237575,1309504,Diallo,Sanoussi,Diallo Sanoussi,2024,2778,diallo-sanoussi,NaN,NaN,Cote d'Ivoire,...,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/diallo-sanouss...,DK1,Fodbold Club Nordsjælland,NaN,NaN,None,NaN
237576,1309800,Issaka,Seidu,Issaka Seidu,2024,2778,issaka-seidu,NaN,NaN,Ghana,...,NaN,CAA Stellar,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/issaka-seidu/p...,DK1,Fodbold Club Nordsjælland,NaN,NaN,None,NaN


In [118]:
ronaldo = players_df.loc[players_df['name'].str.contains('Cristiano Ronaldo')]
# ronaldo
# def get_appearnces_list(game_id):
#     return res_df 
# # appearances_df.tail(5)
# get_appearnces_list(4451012)

    # res_df = appearances_df.loc[appearances_df['appearance_id'].str.game_id]
# appeared_players_df = appearances_df.loc[appearances_df['game_id'] == game_id]
def is_enough_data_to_init_elo(player_id, game_id):
    """
    Decide if we should use
    1. Squad's avg elo
    2. Player's market value of that time
    to init. player's elo
    Assume. Player's elo should be initialised.
    @param player_id: 
    @param game_id: 
    @param season: 
    @return: 
    """
    teammates = appearances_df.loc[(appearances_df['game_id'] == game_id) & (appearances_df['player_id'] != player_id)]
    # This might go wrong?
    season = games_df.loc[games_df['game_id'] == game_id, 'season'].iloc[0]
    if teammates.empty:
        # No teammate at all, so probs just use mrkt value
        return None
    else:
        # Teammate exists. More than half's elo exists?
        res = []

        for player_id in teammates['player_id']:
            elo = players_elo_df.loc[(players_elo_df['player_id'] == player_id) & (players_elo_df['season'] == season)]
            if elo is not None:
                res.append(elo)
                
        if len(res) >= len(teammates) / 2:
            # More than half of teammates' elo exists, so we just avg them
            return None
        else:
            return sum(res) / len(res)

from scipy.stats import zscore
BASE_ELO = 1500
ELO_RANGE = 300
def init_player_elo_with_player_value(player_id, season, base_elo=BASE_ELO, elo_range=ELO_RANGE):
    """
    Init. player's elo based on a player value of that time.
    Assume: His teammates didn't have much data as well...
    @param player_id: 
    @param season: 
    """
    player_value = player_valuations_df.loc[(player_valuations_df['player_id'] == player_id) \
                                            & (player_valuations_df['season'] == season)]
    
    if player_value.empty:
        # So we have no value of that player at that season..
        # TODO: Better value for very first initialisation 
        return 1500
    else:
        season_df = player_valuations_df[player_valuations_df['season'] == season].copy()
        # Normalise
        # Check if we have enough data to calculate mean and std
        if len(season_df['market_value'].dropna()) > 1:
            # Calculate z-scores for market values
            season_df['market_value_z'] = zscore(season_df['market_value'].fillna(season_df['market_value'].mean()))
        else:
            season_df['market_value_z'] = 0  # Default to zero if only one player or no valid market values
        
        # Map z-scores to ELO
        season_df['elo'] = base_elo + (season_df['market_value_z'] * (elo_range / 2))
        
        # Handle cases with missing market value (assign base ELO or another strategy)
        season_df['elo'].fillna(base_elo, inplace=True)
        
        # Merge back to main DataFrame
        players_elo_df = players_elo_df.merge(season_df[['player_id', 'season', 'elo']], on=['player_id', 'season'], how='left')
        
        return players_elo_df
        

    
    
def init_player_elo(player_id, game_id):
    """
    Init player's elo of player id, at game_id
    @param player_id: 
    @param game_id: 
    @return: 
    """
    # 1. Get player's club (at that time) 
    player_appearance = appearances_df.loc[(appearances_df['game_id'] == game_id) \
                                           & (appearances_df['player_id'] == player_id)]
    if player_appearance.empty:
        # Error: No such player
        raise ValueError("No result found for player {} in game {}".format(player_id, game_id))
    club = player_appearance['player_club_id']
    
    # 2. and check if we have enough elo data of that club
    elo_value = is_enough_data_to_init_elo(player_id, game_id)
    # season = games_df.loc[games_df['game_id'] == game_id]['season']
    season = games_df.loc[games_df['game_id'] == game_id, 'season'].iloc[0]
    if elo_value is None:
        # We need to manually calculate elo_value based on his market value of taht time
        # print(season)
        elo_value = init_player_elo_with_player_value(player_id, season)
        
    print(f"Elo value {elo_value} for player {player_id}")
    # 3. Now set player elo of that time
    players_elo_df.loc[(players_elo_df['player_id'] == player_id) \
                       & (players_elo_df['season'] == season), 'elo'] = elo_value
    # NOTE: We can also add feature like looking up player's elo of prev / next season.    
    # Note, that, since we chronologically loop through games, his teammate's elo will be elo of that time!
    # However, be careful finding out which club he was playing for at THAT time.
    # return

def get_player_elo(player_id, game_id):
    """
    Get player {player_id} elo, and if elo is not initialised, it will initialise the player's elo.
    @param player_id: player's id
    @return: player['elo'], integer
    """
    player: pd.DataFrame = players_df.loc[players_df['player_id'] == player_id]
    # If there is multiple....
    if len(player) > 1:
        raise ValueError(f"Multiple results found for player {player_id}. Expected only one")
    elif len(player) == 0:
        raise ValueError(f"No results found for player {player_id}")
    
    # Nothing went wrong
    elo_value = player.get('elo')
    if elo_value is not None:
      # Elo value exists, do something with it
        print(f"Elo value exists for player {player_id}!")
        return player['elo']
    else:
      # Handle the case where 'elo' column doesn't exist or is empty
      # print("Player DataFrame doesn't have an 'elo' column or values are empty")
        print(f"Empty elo_value for player {player_id}, initialising.")
        return init_player_elo(player_id, game_id)

    # if 'elo' in player.columns & player['elo'] is not None:
    #     # Elo alr exist
    # else:
    #     # Elo doesn't exist
    #     



In [114]:
sample_df = appearances_df.head(10)
for index, col in sample_df.iterrows():
    print(get_player_elo(col['player_id'], col['game_id']))
    

Empty elo_value for player 38004, initialising.
Elo value 1500 for player 38004
None
Empty elo_value for player 79232, initialising.
Elo value 1500 for player 79232
None
Empty elo_value for player 42792, initialising.
Elo value 1500 for player 42792
None
Empty elo_value for player 73333, initialising.
Elo value 1500 for player 73333
None
Empty elo_value for player 122011, initialising.
Elo value 1500 for player 122011
None
Empty elo_value for player 146889, initialising.
Elo value 1500 for player 146889
None
Empty elo_value for player 28716, initialising.
Elo value 1500 for player 28716
None
Empty elo_value for player 69445, initialising.
Elo value 1500 for player 69445
None
Empty elo_value for player 19409, initialising.
Elo value 1500 for player 19409
None
Empty elo_value for player 30003, initialising.
Elo value 1500 for player 30003
None


In [105]:
players_elo_df[~players_elo_df['elo'].isnull()]

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur,elo,season


In [106]:
players_elo_df

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur,elo,season
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0,None,2004
1,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0,None,2005
2,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0,None,2006
3,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0,None,2007
4,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0,None,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237573,1306131,Fábio,Sambú,Fábio Sambú,2024,2503,fabio-sambu,Portugal,Barreiro,Portugal,...,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/fabio-sambu/pr...,PO1,Boavista Futebol Clube,NaN,NaN,None,NaN
237574,1306851,NaN,Hakim,Hakim,2024,5724,hakim,Ghana,Techiman,Ghana,...,2026-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/hakim/profil/s...,DK1,Randers Fodbold Club,NaN,NaN,None,NaN
237575,1309504,Diallo,Sanoussi,Diallo Sanoussi,2024,2778,diallo-sanoussi,NaN,NaN,Cote d'Ivoire,...,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/diallo-sanouss...,DK1,Fodbold Club Nordsjælland,NaN,NaN,None,NaN
237576,1309800,Issaka,Seidu,Issaka Seidu,2024,2778,issaka-seidu,NaN,NaN,Ghana,...,NaN,CAA Stellar,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/issaka-seidu/p...,DK1,Fodbold Club Nordsjælland,NaN,NaN,None,NaN


In [ ]:

def calculate_player_elo(appearances_df, game_id):
    appeared_players_df: pd.DataFrame = appearances_df.loc[appearances_df['game_id'] == game_id]
    for index, column in appeared_players_df.iterrows():
        # Get list of players who appeared in certain game. 
        player = players_df.loc[players_df['player_id'] == column['player_id']]
        # Get or init. player elo
        player['elo']
        print(player['name'])
    
calculate_player_elo(appearances_df, 2235545)
appearances_df

In [ ]:
# 
# 
# def sort_games_by_date(df: pd.DataFrame) -> pd.DataFrame:
#     """Sort club_games_df by date
#     It handles changing the date to datetime format as well
# 
#     Args:
#         club_games_df (pd.DataFrame): Assume club_games_df have "date"
# 
#     Returns:
#         pd.DataFrame: _description_
#     """
# 
#     # NOTE: This is why it might fail? Chagne to index or columns if this fails
#     if "date" not in df.columns:
#         raise ValueError("The 'date' column does not exist in the DataFrame.")
# 
#     df["date"] = df["date"].str.strip()
#     df["date"] = pd.to_datetime(df["date"], errors="coerce", format="%Y-%m-%d")
#     df.dropna(subset=["date"], inplace=True)
#     # Sort by date
#     df = df.sort_values(by=["date"])
# 
#     return df
# 
# 
# def expected_chance_of_success(home_team_elo, away_team_elo) -> int:
#     """Calculate chance of success E_home
# 
#     Args:
#         home_team_elo (int): _description_
#         away_team_elo (int): _description_
# 
#     Returns:
#         int : Expected chance of home team winning
#     """
#     return 1 / (1 + 10 ** ((away_team_elo - home_team_elo) / 400))
# 
# 
# # def decide_K_value(game: pd.Series):
# #     """Decide K value based on league or etc?
# 
# #     Args:
# #         game (pd.Series): _description_
# #     """
# 
# 
# def calculate_new_elo(clubs_df: pd.DataFrame, game: pd.Series, K=30):
#     """Calculate new elo
# 
#     Args:
#         home_team_elo (_type_): _description_
#         away_team_elo (_type_): _description_
#         result (_type_): result (should be 1, 0, -1)
#         K (int, optional): _description_. Defaults to 30.
# 
#     Returns:
#         tuple (int, int): new elos of home and away team
#     """
#     result = game["is_win"]
# 
#     if result not in [-1, 0, 1]:
#         raise AssertionError("Invalid result value passed!")
# 
#     home_club_id = game["home_club_id"]
#     away_club_id = game["away_club_id"]
# 
#     home_club_elo = clubs_df.loc[clubs_df["club_id"] == home_club_id, "elo"].values[0]
#     away_club_elo = clubs_df.loc[clubs_df["club_id"] == away_club_id, "elo"].values[0]
# 
#     expected_home_team = expected_chance_of_success(home_club_elo, away_club_elo)
#     expected_away_team = 1 - expected_home_team
# 
#     new_home_elo = home_club_elo + K * (result - expected_home_team)
#     new_away_elo = away_club_elo + K * ((1 - result) - expected_away_team)
# 
#     return new_home_elo, new_away_elo
# 
# 
# def clean_games_df(games_df: pd.DataFrame, clubs_df: pd.DataFrame) -> pd.DataFrame:
#     """Clean games_df so that it only contains games between clubs that
#     exists in clubs_df
# 
#     Args:
#         games_df (pd.DataFrame): _description_
#         clubs_df (pd.DataFrame): _description_
# 
#     Returns:
#         pd.DataFrame: _description_
#     """
# 
#     # Clean games_df so that it only contains games that have valid club id (ie. Clubs in our club_df)
#     home_exists = games_df["home_club_id"].isin(clubs_df["club_id"])
#     away_exists = games_df["away_club_id"].isin(clubs_df["club_id"])
#     cleaned_games_df = games_df[home_exists & away_exists]
# 
#     return cleaned_games_df
# 
# 
# def calculate_clubs_elo_single_match(clubs_df: pd.DataFrame, row):
# 
#     home_club_id = row["home_club_id"]
#     away_club_id = row["away_club_id"]
# 
#     # Get current ELOs for home and away clubs from clubs_df
#     home_elo = clubs_df.loc[clubs_df["club_id"] == home_club_id, "elo"].values[0]
#     away_elo = clubs_df.loc[clubs_df["club_id"] == away_club_id, "elo"].values[0]
# 
#     # Get result for the game (assuming 1 for home win, 0 for away win, 0.5 for draw)
#     game_result = row["is_win"]  # Adjust this based on your game logic
# 
#     # Update ELOs based on the result
#     new_home_elo, new_away_elo = calculate_new_elo(row)
# 
#     clubs_df.loc[clubs_df["club_id"] == home_club_id, "elo"] = new_home_elo
#     clubs_df.loc[clubs_df["club_id"] == away_club_id, "elo"] = new_away_elo
# 
#     return clubs_df
# 
# 
# def calculate_clubs_elo():
#     """
#     Basically calculates clubs elo from scratch
# 
#     1. Read csv files (from transfermrkt dataset) to create a dictionary of DataFrames
#     2. Clean games and filter them by date
#     3. Calculate ELO using very basic ELO
# 
#     Returns:
#         pd.DataFrame : clubs_df with updated club ELO
#     """
# 
#     # Import data
#     dataframes = import_data_from_csv()
#     games_df = dataframes["games_df"]
#     clubs_df = dataframes["clubs_df"]
#     club_games_df = dataframes["club_games_df"]
# 
#     cleaned_games_df = clean_games_df(games_df, clubs_df)
# 
#     # Merge
#     club_games_merged_df = pd.merge(
#         cleaned_games_df, club_games_df, on="game_id", how="inner"
#     )
#     
#     club_games_merged_df.to_csv('club_games_merged.csv')
# 
#     # Now, we will sort by date (Oldest to Latest)
#     # To calculate and update ELO by time
#     # Change data type to datetime format
#     club_games_merged_df = sort_games_by_date(club_games_merged_df)
# 
#     # Initialise club elos
#     # NOTE: Here we are using 1500, but better init. can be done like considering team market value
#     # or level of league they are playing for etc
#     clubs_df["elo"] = 1500
#     # Ensure the 'elo' column is of type float
#     clubs_df["elo"] = clubs_df["elo"].astype(float)
#     test_games_df = club_games_merged_df.head(5000)
# 
#     update_counter = 0
#     for idx, row in test_games_df.iterrows():
#         # home_club_id = row["home_club_id"]
#         # away_club_id = row["away_club_id"]
# 
#         # # Get current ELOs for home and away clubs from clubs_df
#         # home_elo = clubs_df.loc[clubs_df["club_id"] == home_club_id, "elo"].values[0]
#         # away_elo = clubs_df.loc[clubs_df["club_id"] == away_club_id, "elo"].values[0]
# 
#         # # Get result for the game (assuming 1 for home win, 0 for away win, 0.5 for draw)
#         # game_result = row["is_win"]  # Adjust this based on your game logic
# 
#         # # Update ELOs based on the result
#         # new_home_elo, new_away_elo = calculate_new_elo(home_elo, away_elo, game_result)
# 
#         # clubs_df.loc[clubs_df["club_id"] == home_club_id, "elo"] = new_home_elo
#         # clubs_df.loc[clubs_df["club_id"] == away_club_id, "elo"] = new_away_elo
#         clubs_df = calculate_clubs_elo_single_match(clubs_df, row)
# 
#         update_counter += 1
# 
#     print(f"ELO was {update_counter} times updated")
# 
#     clubs_df.to_csv("club_elos.csv")
#     return clubs_df
# 
# 
# df = calculate_clubs_elo()
# print(df.head())
# print(df.tail())